In [85]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import math
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_percentage_error
import holidays
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import optuna

def mean_absolute_percentage_error(y_true, y_pred):
    return (abs(y_true - y_pred) / y_true).mean() * 100


df = pd.read_csv("monthly_data.csv")

In [86]:
import pandas as pd

# Assuming your dataframe is named 'df'
mapping = {
    120004096: "Product1",
    120009814: "Product2",
    120009816: "Product3",
    120010342: "Product4",
    120010566: "Product5",
    120010970: "Product6",
    120011556: "Product7",
    120011782: "Product8",
    120012154: "Product9",
    120012606: "Product10",
    120014486: "Product11",
    120014488: "Product12",
    120015842: "Product13",
    120015996: "Product14"
}

df['ID_material'] = df['ID_material'].map(mapping)

In [87]:
df = df[df["ID_material"] == "Product3"]
df = df.drop(columns="ID_material")
df.head(3)

,Year,Season,MonthName,Units,Number_Of_Holidays_In_That_Month_SPAIN,GDP_Past_Quarter_Change,PreviousUnemploymentQuarterAmmount,Months_Since_Material_Release,Absolute_Previous GDP Quarter Change
26,2018,Winter,February,2160.0,0,0.7,3766.7,0,0.7
30,2018,Spring,March,288.0,1,0.7,3766.7,1,0.7
33,2018,Spring,April,3744.0,1,0.5,3796.1,2,0.5


In [88]:
categorical_cols = ['MonthName',"Season"]
numerical_cols = ["Number_Of_Holidays_In_That_Month_SPAIN","GDP_Past_Quarter_Change","PreviousUnemploymentQuarterAmmount","Months_Since_Material_Release","Absolute_Previous GDP Quarter Change"]
# Filter the data for the last month of 2022 (August)
test_df = df[(df['Year'] == 2022) & (df['MonthName'].isin(['September','August']))]
val_df =df[(df['Year'] == 2022) & (df['MonthName'].isin(['June','July']))]

In [89]:
# Separate the features and target variable for the test set
X_test = test_df.drop(['Units'], axis=1)
y_test = test_df['Units']
y_val = val_df["Units"]

# Filter the data for training (excluding the last month of 2022)
train_df = df[~((df['Year'] == 2022) & (df['MonthName'].isin(['September','August','June','July'])))]

# Separate the features and target variable for training
X_train = train_df.drop(['Units'], axis=1)
y_train = train_df['Units']

X_val = val_df.drop(['Units'], axis=1)
y_val = val_df['Units']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_cols),
        ('num', StandardScaler(), numerical_cols)
    ])

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)
X_val_processed = preprocessor.transform(X_val)

In [90]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.001, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.001, 1.0),
        'random_state': 42
    }

    regressor = xgb.XGBRegressor(**params)
    regressor.fit(X_train_processed, y_train)
    predictions = regressor.predict(X_val_processed)
    score = mean_absolute_percentage_error(y_val.sum(), predictions.sum())  # Replace with your actual scoring function
    return score

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

best_params = study.best_params
best_score = study.best_value

[I 2023-06-22 18:50:27,606] A new study created in memory with name: no-name-c732fe23-d218-4b71-96f5-198801e44219
C:\Users\Usuario\AppData\Local\Temp\ipykernel_7140\2967198301.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
C:\Users\Usuario\AppData\Local\Temp\ipykernel_7140\2967198301.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\Usuario\AppData\Local\Temp\ipykernel_7140\2967198301.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://

In [91]:
import pandas  as pd
import numpy   as np
import xgboost as xgb

regressor=xgb.XGBRegressor(**best_params)
regressor.fit(X_train_processed, y_train)
predictions = regressor.predict(X_test_processed)

In [92]:
predictions.sum()

3799.5073

In [93]:
y_test.sum()

5472.0

In [94]:
mean_absolute_percentage_error(y_test.sum(),predictions.sum())

30.564559133429274